In [1]:
!pip install bert_score

import torch
import pandas as pd
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration

test_df = pd.read_csv("test_processed.csv")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.3 MB/s eta 0:00:00


In [2]:
tokenizer_t5 = T5Tokenizer.from_pretrained("google-t5/t5-large")

device = torch.device("cuda") # GPU usage
model_t5 = T5ForConditionalGeneration.from_pretrained("google-t5/t5-large")
model_t5.to(device)

model_t5

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:246: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on google-t5/t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4096, out_features=1024, bias=False)
              (d

In [3]:
MAX_ALLOWED_SEQUENCE = 8192

In [4]:
def GoBackReadingWindow(model, tokenizer, text):
    torch.cuda.empty_cache()
    if len(text.split()) < MAX_ALLOWED_SEQUENCE:
        inputs = tokenizer("summarize: " + text,\
                           return_tensors="pt",\
                           max_length=MAX_ALLOWED_SEQUENCE,\
                           truncation=True).to(device)
        outputs = model.generate(**inputs, min_length=0, max_new_tokens=MAX_ALLOWED_SEQUENCE)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

    OVERLAP = int(MAX_ALLOWED_SEQUENCE * 0.2)
    words = text.split()
    chunks = []
    start = 0

    while start < len(words):
        end = min(start + MAX_ALLOWED_SEQUENCE, len(words))
        chunks.append(' '.join(words[start:end]))
        start += MAX_ALLOWED_SEQUENCE - OVERLAP

    print(len(chunks))

    summarized_chunks = []
    for chunk in chunks:
        inputs = tokenizer("summarize: " + chunk,\
                           return_tensors="pt",\
                           max_length=MAX_ALLOWED_SEQUENCE,\
                           truncation=True).to(device)
        outputs = model.generate(**inputs, min_length=0, max_new_tokens=MAX_ALLOWED_SEQUENCE)
        summarized_chunks.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
        torch.cuda.empty_cache()

    summarized_text = ' '.join(summarized_chunks)

    if len(summarized_text.split()) > MAX_ALLOWED_SEQUENCE:
        return GoBackReadingWindow(model, tokenizer, summarized_text)
    else:
        inputs = tokenizer("summarize: " + summarized_text,\
                           return_tensors="pt",\
                           max_length=MAX_ALLOWED_SEQUENCE,\
                           truncation=True).to(device)
        outputs = model.generate(**inputs, min_length=0, max_new_tokens=MAX_ALLOWED_SEQUENCE)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [5]:
def ReadingAfterwardsWindow(model, tokenizer, text):
    torch.cuda.empty_cache()
    if len(text.split()) < MAX_ALLOWED_SEQUENCE:
        inputs = tokenizer("summarize: " + text,\
                           return_tensors="pt",\
                           max_length=MAX_ALLOWED_SEQUENCE,\
                           truncation=True).to(device)
        outputs = model.generate(**inputs, min_length=0, max_new_tokens=MAX_ALLOWED_SEQUENCE)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

    OVERLAP = int(MAX_ALLOWED_SEQUENCE * 0.2)
    words = text.split()
    chunks = []
    start = 0

    isfirts = True
    while start < len(words):
        end = min(start + MAX_ALLOWED_SEQUENCE, len(words))
        if isfirts:
            chunks.append(' '.join(words[start:end]))
            isfirts = False
        else:
            chunks.append(' '.join(words[start + OVERLAP:end]) + " " + ' '.join(words[start:start + OVERLAP]))
        start += MAX_ALLOWED_SEQUENCE - OVERLAP

    print(len(chunks))

    summarized_chunks = []
    for chunk in chunks:
        inputs = tokenizer("summarize: " + chunk,\
                           return_tensors="pt",\
                           max_length=MAX_ALLOWED_SEQUENCE,\
                           truncation=True).to(device)
        outputs = model.generate(**inputs, min_length=0, max_new_tokens=MAX_ALLOWED_SEQUENCE)
        summarized_chunks.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
        torch.cuda.empty_cache()

    summarized_text = ' '.join(summarized_chunks)

    if len(summarized_text.split()) > MAX_ALLOWED_SEQUENCE:
        return ReadingAfterwardsWindow(model, tokenizer, summarized_text)
    else:
        inputs = tokenizer("summarize: " + summarized_text,\
                           return_tensors="pt",\
                           max_length=MAX_ALLOWED_SEQUENCE,\
                           truncation=True).to(device)
        outputs = model.generate(**inputs, min_length=0, max_new_tokens=MAX_ALLOWED_SEQUENCE)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [6]:
def ReadingWithJumpingWindow(model, tokenizer, text):
    torch.cuda.empty_cache()
    if len(text.split()) < MAX_ALLOWED_SEQUENCE:
        inputs = tokenizer("summarize: " + text,\
                           return_tensors="pt",\
                           max_length=MAX_ALLOWED_SEQUENCE,\
                           truncation=True).to(device)
        outputs = model.generate(**inputs, min_length=0, max_new_tokens=MAX_ALLOWED_SEQUENCE)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

    JUMP = 100
    words = text.split()
    chunks = []
    start = 0

    while start < len(words):
        end = min(int(start + MAX_ALLOWED_SEQUENCE + JUMP), len(words))
        chunks.append(' '.join(words[int(start):int(start + MAX_ALLOWED_SEQUENCE/2)]) \
                        + " " + ' '.join(words[int(start + MAX_ALLOWED_SEQUENCE/2) + JUMP:end]))
        start += MAX_ALLOWED_SEQUENCE/2

    print(len(chunks))

    summarized_chunks = []
    for chunk in chunks:
        inputs = tokenizer("summarize: " + chunk,\
                           return_tensors="pt",\
                           max_length=MAX_ALLOWED_SEQUENCE,\
                           truncation=True).to(device)
        outputs = model.generate(**inputs, min_length=0, max_new_tokens=MAX_ALLOWED_SEQUENCE)
        summarized_chunks.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
        torch.cuda.empty_cache()

    summarized_text = ' '.join(summarized_chunks)

    if len(summarized_text.split()) > MAX_ALLOWED_SEQUENCE:
        return ReadingWithJumpingWindow(model, tokenizer, summarized_text)
    else:
        inputs = tokenizer("summarize: " + summarized_text,\
                           return_tensors="pt",\
                           max_length=MAX_ALLOWED_SEQUENCE,\
                           truncation=True).to(device)
        outputs = model.generate(**inputs, min_length=0, max_new_tokens=MAX_ALLOWED_SEQUENCE)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [7]:
from bert_score import score

F1_t51 = []

for index, row in test_df.iterrows():
    reference_text = row["reference"]
    reference_summary = row["summary"]
    print(row["reference_tokens_preprocessed"])

    result_summary = GoBackReadingWindow(model_t5, tokenizer_t5, reference_text)
    print(result_summary)
    P, R, F1 = score([result_summary], [reference_summary], lang='en', verbose=False)
    print(f"T5 BertScore F1: {F1.item():.2f}")
    F1_t51.append(F1.item())
    torch.cuda.empty_cache()

np.save('F1_t51.npy', F1_t51)

sum1 = 0
for _ in F1_t51:
    sum1 += _
print(sum1 / len(F1_t51))

4996
eu regulation on marketing requirement for collective investment undertaking. it also provides for the establishment of a central database on cross-border marketing of aifs and ucits.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

T5 BertScore F1: 0.82
4077
a certificate shall be granted if the product is protected by a basic patent. it shall be granted if the product meets the conditions laid down in this regulation. the certificate shall be granted if the product is protected by a basic patent. it shall be granted if the product is protected by a valid basic patent. the certificate shall be granted if the product meets the conditions laid down in this regulation.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.80
16042
3
a judgment given in a state bound by this convention shall be enforced in another state bound by this convention. a court of the state of origin shall not be deemed to have jurisdiction. the court of the state of origin shall apply the law of the state of origin.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.78
0.8125093505981772


In [8]:
F1_t52 = []

for index, row in test_df.iterrows():
    reference_text = row["reference"]
    reference_summary = row["summary"]
    print(row["reference_tokens_preprocessed"])

    result_summary = ReadingAfterwardsWindow(model_t5, tokenizer_t5, reference_text)
    print(result_summary)
    P, R, F1 = score([result_summary], [reference_summary], lang='en', verbose=False)
    print(f"T5 BertScore F1: {F1.item():.2f}")
    F1_t52.append(F1.item())
    torch.cuda.empty_cache()

np.save('F1_t52.npy', F1_t52)

sum2 = 0
for _ in F1_t52:
    sum2 += _
print(sum2 / len(F1_t52))

4996
eu regulation on marketing requirement for collective investment undertaking. it also provides for the establishment of a central database on cross-border marketing of aifs and ucits.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.83
14373
3
market surveillance authority shall ensure that product comply with union harmonisation legislation. it shall ensure that it is made available on the market in a manner that is consistent with the legislation. eu regulations shall apply to products imported into the eu from a third country.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.81
2073
eu directive aims to enhance cross-border access to a greater number of television and radio programme. it lay down rule for the transmission of television and radio programme through the process of direct injection. member state shall ensure that rightholders may exercise their right to grant or refuse the authorisation for a retransmission.


...

T5 BertScore F1: 0.80
4077
a certificate shall be granted if the product is protected by a basic patent. it shall be granted if the product meets the conditions laid down in this regulation. the certificate shall be granted if the product is protected by a basic patent. it shall be granted if the product is protected by a valid basic patent. the certificate shall be granted if the product meets the conditions laid down in this regulation.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.80
16042
3
a court of the state of origin shall not be deemed to have jurisdiction. a court of justice of the european community shall apply this convention. the court of justice of the uk shall not be able to enforce the judgment. the convention shall be open for ratification by the signatory state.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.80
0.8126828284824595


In [9]:
F1_t53 = []

for index, row in test_df.iterrows():
    reference_text = row["reference"]
    reference_summary = row["summary"]
    print(row["reference_tokens_preprocessed"])

    result_summary = ReadingWithJumpingWindow(model_t5, tokenizer_t5, reference_text)
    print(result_summary)
    P, R, F1 = score([result_summary], [reference_summary], lang='en', verbose=False)
    print(f"T5 BertScore F1: {F1.item():.2f}")
    F1_t53.append(F1.item())
    torch.cuda.empty_cache()

np.save('F1_t53.npy', F1_t53)

sum3 = 0
for _ in F1_t53:
    sum3 += _
print(sum3 / len(F1_t53))

4996
eu regulation on marketing requirement for collective investment undertaking. it also provides for the establishment of a central database on cross-border marketing of aifs and ucits.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.83
14373
4
market surveillance authority shall ensure that product comply with union harmonisation legislation. it shall do this by ensuring that it is made available on the market in a manner that is consistent with the legislation.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.80
2073
eu directive aims to enhance cross-border access to a greater number of television and radio programme. it lay down rule for the transmission of television and radio programme through the process of direct injection. member state shall ensure that rightholders may exercise their right to grant or refuse the authorisation for a retransmission.


...

T5 BertScore F1: 0.79
4077
a certificate shall be granted if the product is protected by a basic patent. it shall be granted if the product meets the conditions laid down in this regulation. the certificate shall be granted if the product is protected by a basic patent. it shall be granted if the product is protected by a valid basic patent. the certificate shall be granted if the product meets the conditions laid down in this regulation.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.80
16042
4
a court of a state bound by this convention may refuse to recognise a judgment given in another state. a judgment which orders a periodic payment by way of penalty shall be enforceable in another state. the convention shall be open for ratification by the signatory.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.79
0.8124832632069919
